# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [52]:

import datetime 
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder



In [53]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [54]:
portfolio.head(10)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [55]:
# rename column
portfolio.rename(columns={'id': 'offer_id'}, inplace=True)


In [56]:
# converting days into hours
portfolio['duration_hour'] = portfolio['duration']*24

In [57]:
# Work with categorical variables 'chanels'  
# For us, it is not essential 'email' as it is in all offers
#portfolio['email'] = portfolio['channels'].apply(lambda x: 1 if 'email' in  x else 0)
portfolio['mobile'] = portfolio['channels'].apply(lambda x: 1 if 'mobile' in  x else 0)
portfolio['web'] = portfolio['channels'].apply(lambda x: 1 if 'web' in  x else 0)
portfolio['social'] = portfolio['channels'].apply(lambda x: 1 if 'social' in  x else 0)
# drop the old column
portfolio.drop(['channels'], axis=1, inplace=True )

In [58]:
# Work with categorical variables 'offer_type'
# Create OneHotEncoder object
encoder = OneHotEncoder()

# Perform one-hot encoding on the categorical variable
encoded_data = encoder.fit_transform(portfolio[['offer_type']])

# Convert sparse matrix to dense format and create a new DataFrame
onehot_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.categories_[0])

# Concatenate the original DataFrame and the new DataFrame with encoded features
portfolio = pd.concat([portfolio, onehot_df], axis=1)

# drop the old column
portfolio.drop(['offer_type'], axis=1, inplace=True )


In [59]:
portfolio.head()

,reward,difficulty,duration,offer_id,duration_hour,mobile,web,social,bogo,discount,informational
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,168,1,0,1,1.0,0.0,0.0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,120,1,1,1,1.0,0.0,0.0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,96,1,1,0,0.0,0.0,1.0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,168,1,1,0,1.0,0.0,0.0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,240,0,1,0,0.0,1.0,0.0


In [60]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   reward         10 non-null     int64  
 1   difficulty     10 non-null     int64  
 2   duration       10 non-null     int64  
 3   offer_id       10 non-null     object 
 4   duration_hour  10 non-null     int64  
 5   mobile         10 non-null     int64  
 6   web            10 non-null     int64  
 7   social         10 non-null     int64  
 8   bogo           10 non-null     float64
 9   discount       10 non-null     float64
 10  informational  10 non-null     float64
dtypes: float64(3), int64(7), object(1)
memory usage: 1008.0+ bytes


------------------------------------------------------------------------------------------

In [61]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [62]:
# rename column
profile.rename(columns={'id': 'user_id'}, inplace=True)


In [63]:
# converting int into date
date_member = pd.to_datetime(profile['became_member_on'], format="%Y%m%d").dt.strftime("%d.%m.%Y").tolist()
profile['date_member'] =date_member

# drop the old column
profile.drop(['became_member_on'], axis=1, inplace=True )


In [67]:
# Fill None values in the 'gender' column with 'O'
profile['gender'].fillna('O', inplace=True)

In [71]:
# Fill None values in the 'gender' column with 'O'
profile['income'].fillna(0, inplace=True)

In [69]:
# Work with categorical variables 'gender'
# Create OneHotEncoder object
encoder = OneHotEncoder()

# Perform one-hot encoding on the categorical variable
encoded_data = encoder.fit_transform(profile[['gender']])

# Convert sparse matrix to dense format and create a new DataFrame
onehot_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.categories_[0])

# Concatenate the original DataFrame and the new DataFrame with encoded features
profile = pd.concat([profile, onehot_df], axis=1)

# drop the old column
profile.drop(['gender'], axis=1, inplace=True )

In [74]:
profile.head(100)

,age,user_id,income,date_member,F,M,O
0,118,68be06ca386d4c31939f3a4f0e3dd783,0.0,12.02.2017,0.0,0.0,1.0
1,55,0610b486422d4921ae7d2bf64640c50b,112000.0,15.07.2017,1.0,0.0,0.0
2,118,38fe809add3b4fcf9315a9694bb96ff5,0.0,12.07.2018,0.0,0.0,1.0
3,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,09.05.2017,1.0,0.0,0.0
4,118,a03223e636434f42ac4c3df47e8bac43,0.0,04.08.2017,0.0,0.0,1.0
...,...,...,...,...,...,...,...
95,73,0ba6e2ad781c4429afd26cf0b35f2d11,51000.0,01.04.2018,0.0,1.0,0.0
96,118,53bac762af27471eb294ff89ca1bae2c,0.0,06.08.2016,0.0,0.0,1.0
97,50,b5abc1e85be84f43937b4fca0ad238cb,93000.0,06.05.2017,1.0,0.0,0.0
98,90,1c587de019994f93a242c6864fd7bc55,98000.0,10.12.2015,1.0,0.0,0.0


-----

In [75]:
transcript.head(100)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
95,7c8479d8241344f6a0c306621e191c75,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
96,5404741ead9442bdbd4b9aa2273e85af,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0
97,81e9ad5fa55a4884b65e01a5db8f1b4e,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0
98,ed46fca6de7042478b411690878dc069,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


In [76]:
transcript.rename(columns={'person': 'user_id'}, inplace=True)


In [77]:
transcript.groupby('user_id').count()

,event,value,time
user_id,,,
0009655768c64bdeb2e877511632db8f,20,20,20
00116118485d4dfda04fdbaba9a87b5c,7,7,7
0011e0d4e6b944f998e987f904e8c1e5,18,18,18
0020c2b971eb4e9188eac86d93036a77,19,19,19
0020ccbbb6d84e358d3414a3ff76cffd,23,23,23
...,...,...,...
fff3ba4757bd42088c044ca26d73817a,23,23,23
fff7576017104bcc8677a8d63322b5e1,18,18,18
fff8957ea8b240a6b5e634b6ee8eafcf,10,10,10


In [78]:
transcript.groupby('event').count()

,user_id,value,time
event,,,
offer completed,33579,33579,33579
offer received,76277,76277,76277
offer viewed,57725,57725,57725
transaction,138953,138953,138953


In [79]:
df_transcript = transcript[transcript['event'] == 'transaction'].copy()
df_transcript['transaction_value'] = [float(val['amount']) for val in df_transcript['value']]
df_transcript['offer_id'] = [0]*len(df_transcript)
df_transcript['completed_value'] = [0]*len(df_transcript)

print(df_transcript.shape)
df_transcript.head()



(138953, 7)


,user_id,event,value,time,transaction_value,offer_id,completed_value
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0,0.83,0,0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0,34.56,0,0
12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0,13.23,0,0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0,19.51,0,0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0,18.97,0,0


In [80]:
df_transcript[['user_id','transaction_value']].groupby('user_id').sum()

,transaction_value
user_id,
0009655768c64bdeb2e877511632db8f,127.60
00116118485d4dfda04fdbaba9a87b5c,4.09
0011e0d4e6b944f998e987f904e8c1e5,79.46
0020c2b971eb4e9188eac86d93036a77,196.86
0020ccbbb6d84e358d3414a3ff76cffd,154.05
...,...
fff3ba4757bd42088c044ca26d73817a,580.98
fff7576017104bcc8677a8d63322b5e1,29.94
fff8957ea8b240a6b5e634b6ee8eafcf,12.15


In [81]:
df_comleted = transcript[transcript['event']=="offer completed"].copy()
df_comleted['transaction_value'] = [0]*len(df_comleted)
df_comleted['offer_id'] = [val['offer_id'] for val in df_comleted['value']]
df_comleted['completed_value'] = [float(val['reward']) for val in df_comleted['value']]


print(df_comleted.shape)
df_comleted.head(100)

(33579, 7)


,user_id,event,value,time,transaction_value,offer_id,completed_value
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0,0,fafdcd668e3743c1bb461111dcafc2a4,2.0
12679,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
12692,676506bad68e4161b9bbaffeb039626b,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,0,0,ae264e3637204a6fb9bb56bc8210ddfd,10.0
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
...,...,...,...,...,...,...,...
13948,a05d5de8d4314ec19162578f67af3fee,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0
13953,33834c812b4e47e4955093ad03f6117a,offer completed,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
13965,c0231649f05d40889e3a6e1172303b37,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
13976,3b84fac9ca1148bb96b133ced98f565a,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,0,0,f19421c1d4aa40978ebb69ca19b0e20d,5.0


In [19]:
df_comleted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33579 entries, 12658 to 306527
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            33579 non-null  object 
 1   event              33579 non-null  object 
 2   value              33579 non-null  object 
 3   time               33579 non-null  int64  
 4   transaction_value  33579 non-null  int64  
 5   offer_id           33579 non-null  object 
 6   completed_value    33579 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.0+ MB


In [82]:
df_comleted[['user_id','completed_value']].groupby('user_id').sum()

,completed_value
user_id,
0009655768c64bdeb2e877511632db8f,9.0
0011e0d4e6b944f998e987f904e8c1e5,13.0
0020c2b971eb4e9188eac86d93036a77,14.0
0020ccbbb6d84e358d3414a3ff76cffd,13.0
003d66b6608740288d6cc97a6903f4f0,9.0
...,...
fff29fb549084123bd046dbc5ceb4faa,47.0
fff3ba4757bd42088c044ca26d73817a,9.0
fff7576017104bcc8677a8d63322b5e1,9.0


In [21]:
df_viewed = transcript[transcript['event'] == "offer viewed"].copy()
                       
df_viewed['transaction_value'] = [0]*len(df_viewed)
df_viewed['offer_id'] = df_viewed['value'].apply(lambda val: str(val.get('offer id', '')))
df_viewed['completed_value'] = [0]*len(df_viewed)

print(df_viewed.shape)
df_viewed.head(100)

(57725, 7)


,user_id,event,value,time,transaction_value,offer_id,completed_value
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0,0,f19421c1d4aa40978ebb69ca19b0e20d,0
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0,0,5a8bc65990b245e5a138643cd4eb9837,0
12652,102e9454054946fda62242d2e176fdce,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0
12653,02c083884c7d45b39cc68e1314fec56c,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0,0,ae264e3637204a6fb9bb56bc8210ddfd,0
12655,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0,0,5a8bc65990b245e5a138643cd4eb9837,0
...,...,...,...,...,...,...,...
12782,08e66f9936c545d4aedf21d35b96baf6,offer viewed,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,fafdcd668e3743c1bb461111dcafc2a4,0
12787,188540a144304408b5ce19148b295f31,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0
12788,250032cbd1b94b2b822f036f18fb407c,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0
12789,7e45faba0a8346379d0c296f1df8ae1d,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0


In [83]:

# Check the values from df_comleted that are present in df_viewed

df_comleted_tuples = df_comleted[['user_id', 'offer_id']].apply(tuple, axis=1)

# Convert the 'user_id' and 'offer_id' columns of df_viewed to tuples
df_viewed_tuples = df_viewed[['user_id', 'offer_id']].apply(tuple, axis=1)

# Check if the tuples from df_comleted are present in df_viewed
is_present_mask = df_comleted_tuples.isin(df_viewed_tuples)

# Filter df_comleted based on the is_present_mask
df_is = df_comleted[is_present_mask]
df_not = df_comleted[~is_present_mask]

print("Values present in df_viewed:")
print(len(df_is))
print("Values not present in df_viewed:")
print(len(df_not))


Values present in df_viewed:
28724
Values not present in df_viewed:
4855


In [84]:
df_is.head()

,user_id,event,value,time,transaction_value,offer_id,completed_value
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0,0,fafdcd668e3743c1bb461111dcafc2a4,2.0
12679,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
12692,676506bad68e4161b9bbaffeb039626b,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,0,0,ae264e3637204a6fb9bb56bc8210ddfd,10.0
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0


In [85]:
df_not['offer_works'] = [0]*len(df_not)
df_is.loc[:, 'offer_works'] = 1



C:\Users\IlYA\AppData\Local\Temp/ipykernel_11604/2270650960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not['offer_works'] = [0]*len(df_not)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [86]:
df_not.head()

,user_id,event,value,time,transaction_value,offer_id,completed_value,offer_works
12780,b860d355ef6e4c66b5d5a837c56ef32d,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,0
12786,24115a61df25473e84a8a03f3c98de1a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,0
12826,2481f1fcfbcb4b288e5a03af02d95373,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,0
12936,16dc4306258b4fef90ffe99f53ed7952,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,0
12943,2c9a9137efff4820a47d56dbf0fa555f,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,0,0,f19421c1d4aa40978ebb69ca19b0e20d,5.0,0


In [87]:
df_viewed_offer= pd.concat([df_is, df_not], ignore_index=True)
df_viewed_offer.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33579 entries, 0 to 33578
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            33579 non-null  object 
 1   event              33579 non-null  object 
 2   value              33579 non-null  object 
 3   time               33579 non-null  int64  
 4   transaction_value  33579 non-null  int64  
 5   offer_id           33579 non-null  object 
 6   completed_value    33579 non-null  float64
 7   offer_works        33579 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 2.0+ MB


In [86]:
df_tmp = df_transcript.groupby('user_id')['transaction_value'].sum().reset_index()
df_tmp[['user_id', 'transaction_value']].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16578 entries, 0 to 16577
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            16578 non-null  object 
 1   transaction_value  16578 non-null  float64
dtypes: float64(1), object(1)
memory usage: 259.2+ KB


In [98]:
#df_viewed_offer = pd.merge(df_viewed_offer, df_tmp, on='user_id')
#df_viewed_offer.drop('transaction_value_y', axis=1, inplace=True)
#df_viewed_offer.drop('transaction_value_x', axis=1, inplace=True)


df_viewed_offer.shape

(33579, 8)

In [99]:
len(df_viewed_offer['user_id'].unique())

12774

-----

In [56]:
df_received = transcript[transcript['event'] == "offer received"].copy()
                       
df_received['transaction_value'] = [0]*len(df_received)
df_received['offer_id'] = df_received['value'].apply(lambda val: str(val.get('offer id', '')))
df_received['completed_value'] = [0]*len(df_received)

print(df_received.shape)
df_received.head(100)

(76277, 7)


,user_id,event,value,time,transaction_value,offer_id,completed_value
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,2906b810c7d4411798c6938adc9daaa5,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,fafdcd668e3743c1bb461111dcafc2a4,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0
...,...,...,...,...,...,...,...
95,7c8479d8241344f6a0c306621e191c75,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0,0,5a8bc65990b245e5a138643cd4eb9837,0
96,5404741ead9442bdbd4b9aa2273e85af,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0,0,2298d6c36e964ae4a3e7e9706d1fb8c2,0
97,81e9ad5fa55a4884b65e01a5db8f1b4e,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0,0,ae264e3637204a6fb9bb56bc8210ddfd,0
98,ed46fca6de7042478b411690878dc069,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,2906b810c7d4411798c6938adc9daaa5,0


In [132]:
merged_df = pd.merge(df_comleted, portfolio, on='offer_id')

merged_df.shape


(33579, 12)

In [133]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33579 entries, 0 to 33578
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            33579 non-null  object 
 1   event              33579 non-null  object 
 2   value              33579 non-null  object 
 3   time               33579 non-null  int64  
 4   transaction_value  33579 non-null  int64  
 5   offer_id           33579 non-null  object 
 6   completed_value    33579 non-null  float64
 7   reward             33579 non-null  int64  
 8   channels           33579 non-null  object 
 9   difficulty         33579 non-null  int64  
 10  duration           33579 non-null  int64  
 11  offer_type         33579 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 3.3+ MB


In [137]:
merged_df[merged_df['time']==0]

,user_id,event,value,time,transaction_value,offer_id,completed_value,reward,channels,difficulty,duration,offer_type
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,2,"[web, email, mobile]",10,7,discount
1,73ffefd41e9a4ca3ab26b2b3697c6eb7,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,2,"[web, email, mobile]",10,7,discount
2,24115a61df25473e84a8a03f3c98de1a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,2,"[web, email, mobile]",10,7,discount
3,2481f1fcfbcb4b288e5a03af02d95373,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,2,"[web, email, mobile]",10,7,discount
4,a45b69f1c8554ae7af83e74426ca437a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,2,"[web, email, mobile]",10,7,discount
...,...,...,...,...,...,...,...,...,...,...,...,...
30159,89471970d2fc4f5ebe1712424467d0b5,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,5,"[web, email]",20,10,discount
30160,2170ec957f354a51a6e41ecd1a9a8f36,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,5,"[web, email]",20,10,discount
30161,5d3f09e7ccb5466d875925d63db35f72,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,5,"[web, email]",20,10,discount
30162,b9dd94b4eeac44c2b7e3127256772052,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,5,"[web, email]",20,10,discount


In [ ]:
------------------

In [154]:
df_other = transcript[(transcript['event'] == "offer received") | (transcript['event'] == "offer viewed")].copy()
df_other['transaction_value'] = [0]*len(df_other)
df_other['offer_id'] = df_other['value'].apply(lambda val: str(val.get('offer id', '')))
df_other['completed_value'] = [0]*len(df_other)

print(df_other.shape)
df_other.head(100)



(134002, 7)


,user_id,event,value,time,transaction_value,offer_id,completed_value
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,2906b810c7d4411798c6938adc9daaa5,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,fafdcd668e3743c1bb461111dcafc2a4,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0
...,...,...,...,...,...,...,...
95,7c8479d8241344f6a0c306621e191c75,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0,0,5a8bc65990b245e5a138643cd4eb9837,0
96,5404741ead9442bdbd4b9aa2273e85af,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0,0,2298d6c36e964ae4a3e7e9706d1fb8c2,0
97,81e9ad5fa55a4884b65e01a5db8f1b4e,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0,0,ae264e3637204a6fb9bb56bc8210ddfd,0
98,ed46fca6de7042478b411690878dc069,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,2906b810c7d4411798c6938adc9daaa5,0


In [102]:
df = pd.concat([df_comleted, df_other], ignore_index=True)

df = pd.concat([df, df_transcript], ignore_index=True)

print(df.shape)


(306534, 7)


In [103]:
df.drop('value', axis=1, inplace=True)
print(df.shape)


(306534, 6)


In [108]:
df['offer_received'] = df['event'].apply(lambda x: 1 if (x == 'offer received') or (x == 'offer viewed') else 0)
df['offer_viewed'] = df['event'].apply(lambda x: 1 if x == 'offer viewed' else 0)

df['offer_completed'] = df['event'].apply(lambda x: 1 if x == 'offer completed' else 0)
df['transaction'] = df['event'].apply(lambda x: 1 if x == 'transaction' else 0)



In [112]:
df[(df.offer_viewed == 1) & (df.offer_completed == 1)]



,person,event,time,transaction_value,offer_id,completed_value,offer_received,offer_completed,offer_viewed,transaction


In [159]:
common_values = pd.merge(df_comleted, df_other, on=['user_id', 'offer_id'])
common_values.head(100)

,user_id,event_x,value_x,time_x,transaction_value_x,offer_id,completed_value_x,event_y,value_y,time_y,transaction_value_y,completed_value_y
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,0
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0,0,2906b810c7d4411798c6938adc9daaa5,2.0,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,0
2,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0,0,fafdcd668e3743c1bb461111dcafc2a4,2.0,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,0
3,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0,0,fafdcd668e3743c1bb461111dcafc2a4,2.0,offer viewed,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},6,0,0
4,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,bf7a630d566f46f79b855990daf2d0b0,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,0,0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},504,0,0
96,bf7a630d566f46f79b855990daf2d0b0,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,186,0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0,0,0
97,bf7a630d566f46f79b855990daf2d0b0,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,186,0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,offer viewed,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0,0,0
98,bf7a630d566f46f79b855990daf2d0b0,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,186,0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},168,0,0


In [160]:
common_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84104 entries, 0 to 84103
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              84104 non-null  object 
 1   event_x              84104 non-null  object 
 2   value_x              84104 non-null  object 
 3   time_x               84104 non-null  int64  
 4   transaction_value_x  84104 non-null  int64  
 5   offer_id             84104 non-null  object 
 6   completed_value_x    84104 non-null  float64
 7   event_y              84104 non-null  object 
 8   value_y              84104 non-null  object 
 9   time_y               84104 non-null  int64  
 10  transaction_value_y  84104 non-null  int64  
 11  completed_value_y    84104 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 8.3+ MB
